In [1]:
import requests
from pprint import pprint
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
sns.set()
%matplotlib inline

In [2]:
db = pymysql.connect(host='localhost', user='root', database='admin_main', port = 3307, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

# Create a cursor object
cursorObject = db.cursor()

bags_Query = "select * from bags"
orders_Query = "select * from orders"
bagsent_Query =  "select * from bags_entries"
bagsentdet_Query = "select * from bags_entry_details"
items_Query = "select * from items"
itemcat_Query = "select * from item_categories"
ordersent_Query = "select * from orders_entries"
ordersentdet_Query = "select * from orders_entry_details"
orderdeldet_Query = "select * from orders_delivery_details"

cursorObject.execute(bags_Query)
cursorObject.execute(orders_Query)
cursorObject.execute(bagsent_Query)
cursorObject.execute(bagsentdet_Query)
cursorObject.execute(items_Query)
cursorObject.execute(itemcat_Query)
cursorObject.execute(ordersent_Query)
cursorObject.execute(ordersentdet_Query)
cursorObject.execute(orderdeldet_Query)

bags_df = pd.read_sql(bags_Query, db)
orders_df = pd.read_sql(orders_Query, db)
bagsent_df = pd.read_sql(bagsent_Query, db)
bagsentdet_df = pd.read_sql(bagsentdet_Query, db)
items_df = pd.read_sql(items_Query, db)
itemcat_df = pd.read_sql(itemcat_Query, db)
ordersent_df = pd.read_sql(ordersent_Query, db)
ordersentdet_df = pd.read_sql(ordersentdet_Query, db)
orderdeldet_df = pd.read_sql(orderdeldet_Query, db)

In [3]:
bags_new = bags_df.drop(bags_df.columns.difference(['id', 'subscriber_id', 'created_at']), 1)
orders_new = orders_df.drop(orders_df.columns.difference(['id', 'subscriber_id', 'status', 'created_at']), 1)
bagsent_new = bagsent_df.drop(bagsent_df.columns.difference(['id', 'bag_id', 'item_id', 'created_at']), 1)
bagsentdet_new = bagsentdet_df.drop(bagsentdet_df.columns.difference(['bag_entry_id', 'type', 'pos_code', 'quantity']), 1)
items_new = items_df.drop(items_df.columns.difference(['id', 'category_id', 'internal_name']), 1)
itemcat_new = itemcat_df.drop(itemcat_df.columns.difference(['id', 'name']), 1)
                                                                                                                                                          
ordersent_new = ordersent_df.drop(ordersent_df.columns.difference(['id', 'order_id', 'item_id', 'created_at']), 1)
ordersentdet_new = ordersentdet_df.drop(ordersentdet_df.columns.difference(['order_entry_id', 'type', 'pos_code', 'name', 'price', 'quantity', 'created_at']), 1)
orderdeldet_new = orderdeldet_df.drop(orderdeldet_df.columns.difference(['order_id', 'delivery_time']), 1)

In [4]:
bags_new.rename(columns={'id': 'bag_id'}, inplace=True)
orders_new.rename(columns={'id': 'order_id'}, inplace=True)
items_new.rename(columns={'id': 'item_id'}, inplace=True)
itemcat_new.rename(columns={'id': 'category_id'}, inplace=True)
bagsentdet_new.rename(columns={'bag_entry_id': 'id'}, inplace=True)
# bagsentdet = bagsentdet_new[bagsentdet_new.type == 'variance']
ordersentdet_new.rename(columns={'order_entry_id': 'id'}, inplace=True)
# ordersentdet = ordersentdet_new[ordersentdet_new.type == 'variance']
ordersentdet = ordersentdet_new

bags = pd.merge(bagsent_new, bagsentdet_new, on='id', how='inner')
orders = pd.merge(ordersent_new, ordersentdet, on='id', how='inner')
items = pd.merge(items_new, itemcat_new, on='category_id', how='left')
all_bags = pd.merge(bags, items, on='item_id', how='left')
bags2 = pd.merge(all_bags, bags_new, on='bag_id', how='inner')
all_orders = pd.merge(orders, items, on='item_id', how='left')
orders2 = pd.merge(all_orders, orders_new, on='order_id', how='inner')
orders2 = pd.merge(orders2, orderdeldet_new, on='order_id', how='inner')
# users = users_new.groupby('created_at')['id'].nunique()
# subscribers = subscribers_new.groupby('created_at')['id'].nunique()
# orders = orders_new.groupby('created_at')['id'].nunique()
orders2

,id,order_id,item_id,created_at_x,type,pos_code,name_x,price,quantity,created_at_y,category_id,internal_name,name_y,subscriber_id,status,created_at,delivery_time
0,1,1,180,2019-11-13 06:46:56,variance,1520,None,35.0,1,2019-11-13 06:46:56,14.0,Apple Pie,Drinks and Desserts,6,Closed,2019-11-13 06:46:56,2019-11-13 06:46:00
1,2,2,279,2019-11-13 06:48:56,variance,5276,None,95.0,1,2019-11-13 06:48:56,14.0,Classic Milk Tea w/ Brown Sugar Pearls,Drinks and Desserts,5,Closed,2019-11-13 06:48:56,2019-11-13 06:48:00
2,3,3,187,2019-11-13 06:49:25,variance,1754,None,55.0,1,2019-11-13 06:49:25,14.0,McCafé Coffee Float,Drinks and Desserts,4,Closed,2019-11-13 06:49:25,2019-11-13 06:48:00
3,4,4,135,2019-11-13 06:59:40,variance,1645,None,134.0,1,2019-11-13 06:59:40,19.0,Egg McMuffin w/ Hash Browns,Breakfast Sandwiches,8,Closed,2019-11-13 06:59:40,2019-11-13 18:20:00
4,4,4,135,2019-11-13 06:59:40,with_item_variance,1769,Orange Juice,17.0,1,2019-11-13 06:59:40,19.0,Egg McMuffin w/ Hash Browns,Breakfast Sandwiches,8,Closed,2019-11-13 06:59:40,2019-11-13 18:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12003,6165,2919,126,2020-03-02 23:12:20,with_item_variance,5290,Classic Milk Tea w/ Brown Sugar Pearls,40.0,2,2020-03-02 23:12:20,17.0,McSpaghetti w/ Fries,McSpaghetti and Rice Meals,18542,Rider Assigned,2020-03-02 23:12:20,2020-03-02 23:12:00
12004,6165,2919,126,2020-03-02 23:12:20,with_item_variance,8203,Medium Fries,0.0,2,2020-03-02 23:12:20,17.0,McSpaghetti w/ Fries,McSpaghetti and Rice Meals,18542,Rider Assigned,2020-03-02 23:12:20,2020-03-02 23:12:00
12005,6166,2920,34,2020-03-03 00:50:17,variance,1575,None,125.0,1,2020-03-03 00:50:17,7.0,Cheeseburger,Burgers,30490,Rider Out,2020-03-03 00:50:17,2020-03-03 00:49:00
12006,6166,2920,34,2020-03-03 00:50:17,with_item_variance,1783,Coke,0.0,1,2020-03-03 00:50:17,7.0,Cheeseburger,Burgers,30490,Rider Out,2020-03-03 00:50:17,2020-03-03 00:49:00


In [5]:
orders_df['count'] = orders_df.groupby('subscriber_id').cumcount() + 1

In [ ]:
start_date = "2020-03-01 00:00:00"
end_date = "2020-03-31 23:59:59"

orders_df = orders_df[orders_df.created_at.between(start_date,end_date)]
orders2 = orders2[orders2.created_at.between(start_date,end_date)]
bags2 = bags2[bags2.created_at.between(start_date,end_date)]
ordersentdet_new = ordersentdet_new[ordersentdet_new.created_at.between(start_date,end_date)]
bagsentdet_new = bagsentdet_new[bagsentdet_new.created_at.between(start_date,end_date)]

In [6]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/Mc sales by user.xlsx", engine='xlsxwriter')

orders_df.to_excel(writer, sheet_name='Orders')
orders2.to_excel(writer, sheet_name='Orders_det')
bags2.to_excel(writer, sheet_name='Bags_det')

writer.save()

In [7]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/Mc sales.xlsx", engine='xlsxwriter')

ordersentdet_new.to_excel(writer, sheet_name='Orders_det')
bagsentdet_new.to_excel(writer, sheet_name='Bags_det')

writer.save()